# Spills Table

## Packages & Setup

In [2]:
# Get pandas and postgres to work together
import psycopg2 as pg
import pandas as pd
import pandas.io.sql as pd_sql
from datetime import timedelta
import datetime

In [3]:
#Setup Configs
%matplotlib inline
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 25)
pd.set_option('display.precision', 3)

In [4]:
# Postgres info to connect
connection_args = {
    'host': 'localhost',  # We are connecting to our _local_ version of psql
    'dbname': 'col_og',    # DB that we are connecting to
    'port': 5432          # port we opened on AWS
}

# We will talk about this magic Python trick!
connection = pg.connect(**connection_args)

cursor = connection.cursor()

## SQL Functions

In [5]:
def get_tables(cursor):
    query = """
        SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';
        """
    cursor.execute(query)
    table_list = cursor.fetchall()
    table_list_clean = [table[0] for table in table_list]
    return table_list_clean

def get_cols(table,cursor):
    query = """
        SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{}';
        """.format(table)
    cursor.execute(query)
    col_list = cursor.fetchall()
    col_list_clean = [col[0] for col in col_list]
    return col_list_clean

def get_sample(table,connection,limit=10):
    query = """
        SELECT * FROM {} LIMIT {};
        """.format(table,limit)
    df = pd.read_sql(query,connection)
    return df
    
def get_table(table,connection):
    query = """
        SELECT * FROM {};
        """.format(table)
    df = pd.read_sql(query,connection)
    return df

## Spill Cleanup

In [6]:
spill_df = get_table('spills',connection)
spill_df = spill_df.sort_values(['document_#','report','tracking_#'])

spill_df = spill_df.groupby(['tracking_#']).apply(lambda x: x.fillna(axis=0,method='ffill')).reset_index(drop=True)


In [7]:
spill_df = spill_df.drop_duplicates(subset='tracking_#')
spill_df = spill_df.drop(spill_df.index[pd.isnull(spill_df['date_of_discovery'])])

In [8]:
spill_df.index = spill_df['tracking_#']

In [9]:
spill_df.head()

,document_#,report,operator,operator_#,tracking_#,initial_report_date,date_of_discovery,spill_type,qtr_qtr,section,township,range,meridian,latitude,longitude,municipality,county,facility_type,facility_id,api_county_code,api_sequence_number,spilled_outside_of_berms,more_than_five_barrels_spilled,oil_spill_volume,condensate_spill_volume,flow_back_spill_volume,produced_water_spill_volume,e&p_waste_spill_volume,other_waste,drilling_fluid_spill_volume,current_land_use,other_land_use,weather_conditions,surface_owner,surface_owner_other,waters_of_the_state,residence_/_occupied_structure,livestock,public_byway,surface_water_supply_area,spill_description,supplemental_report_date,oil_bbls_spilled,oil_bbls_recovered,oil_unknown,condensate_bbls_spilled,condensate_bbls_recovered,condensate_unknown,produced_water_bbls_spilled,produced_water_bbls_recovered,produced_water_unknown,drilling_fluid_bbls_spilled,drilling_fluid_bbls_recovered,drilling_fluid_unknown,flow_back_fluid_bbls_spilled,flow_back_fluid_bbls_recovered,flow_back_fluid_unkown,other_e&p_waste_bbls_spilled,other_e&p_waste_bbls_recovered,other_e&p_waste_unknown,other_e&p_waste,spill_contained_within_berm,emergency_pit_constructed,soil,groundwater,surface_water,dry_drainage_feature,surface_area_length,surface_area_width,depth_of_impact_in_feet,depth_of_impact_in_inches,area_depth_determined,geology_description,depth_to_groundwater,water_wells_in_area,water_wells,water_wells_none,surface_water_near,surface_water_none,wetlands,wetlands_none,springs,springs_none,livestock_near,livestock_none,occupied_buildings,occupied_buildings_none,additional_spill_details,supplemental_report_date_ca,human_error,equipment_failure,historical_unkown,other,other_description,root_cause,preventative_measures,soil_excavated,offsite_disposal,onsite_treatment,other_disposition,other_disposition_description,ground_water_removed,surface_water_removed,corrective_actions_completed,approved_form_27,form_27_project_number
tracking_#,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
400476365,400476365,I,BP AMERICA PRODUCTION COMPANY ...,10000.0,400476365,08/30/2013,08/30/2013,Recent,SW,36,34N,9W,M,37.144,-107.779,None,LA PLATA,WELL,NaN,067,08262,N,Y,0,0,0,>=5 and <100,0,92 barrels of coal bed methane produced water,0,OTHER,well pad,unknown,FEE,unknown,0.0,0.0,0.0,0.0,0.0,Tank bottom on 300 bbl produced water tank lea...,None,NaN,NaN,False,NaN,NaN,False,NaN,NaN,False,NaN,NaN,False,NaN,NaN,False,NaN,NaN,False,None,NaN,NaN,False,False,False,False,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,False,NaN,False,NaN,False,NaN,False,NaN,False,NaN,False,None,NaT,False,False,False,False,None,None,None,NaN,False,False,False,None,NaN,NaN,False,False,NaN
400579956,400579956,I,ENCANA OIL & GAS (USA) INC ...,100185.0,400579956,03/27/2014,03/26/2014,Recent,SESE,23,1N,68W,6,40.032,-104.965,None,BROOMFIELD,TANK BATTERY,NaN,014,12170,N,N,0,0,0,Unknown,0,None,0,CROP LAND,None,Clear and cool,FEE,NA,1.0,0.0,0.0,0.0,1.0,A high water table at Encana’s Helen E Unit 1 ...,None,NaN,NaN,False,NaN,NaN,False,NaN,NaN,False,NaN,NaN,False,NaN,NaN,False,NaN,NaN,False,None,NaN,NaN,False,False,False,False,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,False,NaN,False,NaN,False,NaN,False,NaN,False,NaN,False,None,NaT,False,False,False,False,None,None,None,NaN,False,False,False,None,NaN,NaN,False,False,NaN
400584981,400584981,I,ENCANA OIL & GAS (USA) INC ...,100185.0,400584981,04/07/2014,03/15/2013,Historical,NESW,6,1N,68W,6,40.077,-105.050,None,WELD,TANK BATTERY,NaN,123,34949,Y,N,Unknown,Unknown,Unknown,Unknown,Unknown,None,Unknown,NON-CROP LAND,None,Clear cold,FEE,None,1.0,0.0,0.0,0.0,0.0,While digging trenches for new dumplines the c...,None,NaN,NaN,False,NaN,NaN,False,NaN,NaN,False,NaN,NaN,False,NaN,NaN,False,NaN,NaN,False,None,NaN,NaN,False,False,False,False,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,False,NaN,False,NaN,False,NaN,False,NaN,False,NaN,False,None,NaT,False,False,False,False,None,None,None,NaN,False,F

In [10]:
spill_df_op = spill_df[['operator','operator_#']]
spill_df_geo = spill_df[['latitude','longitude','county','api_county_code']]

spill_df_strip_predict = spill_df[['oil_spill_volume','condensate_spill_volume','flow_back_spill_volume',
                                    'produced_water_spill_volume','e&p_waste_spill_volume','other_waste',
                                    'drilling_fluid_spill_volume']]

spill_df_incentives = spill_df[['current_land_use','surface_owner','waters_of_the_state',
                                'residence_/_occupied_structure','livestock','public_byway',
                                'surface_water_supply_area']]

spill_df_text = spill_df[['spill_description','root_cause']]

spill_df_info = spill_df[['date_of_discovery','facility_type',
                         'human_error','equipment_failure','historical_unkown','other']]

# 'more_than_five_barrels_spilled'

## Spill Strip Predict

In [11]:
spill_df_strip_predict.head()

,oil_spill_volume,condensate_spill_volume,flow_back_spill_volume,produced_water_spill_volume,e&p_waste_spill_volume,other_waste,drilling_fluid_spill_volume
tracking_#,,,,,,,
400476365,0,0,0,>=5 and <100,0,92 barrels of coal bed methane produced water,0
400579956,0,0,0,Unknown,0,None,0
400584981,Unknown,Unknown,Unknown,Unknown,Unknown,None,Unknown
400596597,0,0,0,>=5 and <100,0,Produced Water (Permitted for Surface Water Di...,0
400597054,0,0,0,Unknown,0,None,0


In [12]:
for col in spill_df_strip_predict.columns:
    print('{} : {}'.format(col,spill_df_strip_predict[col].unique()))

oil_spill_volume : ['0' 'Unknown' '>=1 and <5' '>=5 and <100' '>=100' '>0 and <1']
condensate_spill_volume : ['0' 'Unknown' '>=1 and <5' '>=5 and <100' '>=100' '>0 and <1']
flow_back_spill_volume : ['0' 'Unknown' '>=5 and <100' '>=100' '>=1 and <5' '>0 and <1']
produced_water_spill_volume : ['>=5 and <100' 'Unknown' '>=1 and <5' '0' '>=100' '>0 and <1']
e&p_waste_spill_volume : ['0' 'Unknown' '>=5 and <100' '>=1 and <5' '>0 and <1' '>=100']
other_waste : ['92 barrels of coal bed methane produced water' None
 'Produced Water (Permitted for Surface Water Discharge)'
 'Historic realse discovered during preparatory pit closure investigation'
 'Historic release discovered during pit closure investigation'
 '3 barrels of produced water were released' '5 bbls of produced water'
 '28 bbl of produced water spilled' '30 bbls of produced water'
 '30 bbls were spilled' '100 barrels of produced water were released.'
 'Historical Impacts - Volume of the release is unknown'
 'Historical impacts - Vol

In [13]:
spill_df_strip_predict = spill_df_strip_predict.fillna(0)
for col in spill_df_strip_predict.columns:
    spill_df_strip_predict[col] = pd.to_numeric(spill_df_strip_predict[col],errors='coerce')
spill_df_strip_predict = spill_df_strip_predict.fillna(1)


In [14]:
spill_df_strip_predict.head()

,oil_spill_volume,condensate_spill_volume,flow_back_spill_volume,produced_water_spill_volume,e&p_waste_spill_volume,other_waste,drilling_fluid_spill_volume
tracking_#,,,,,,,
400476365,0.0,0.0,0.0,1.0,0.0,1.0,0.0
400579956,0.0,0.0,0.0,1.0,0.0,0.0,0.0
400584981,1.0,1.0,1.0,1.0,1.0,0.0,1.0
400596597,0.0,0.0,0.0,1.0,0.0,1.0,0.0
400597054,0.0,0.0,0.0,1.0,0.0,0.0,0.0


## Spill Incentives

In [15]:
spill_df_incentives.head()

,current_land_use,surface_owner,waters_of_the_state,residence_/_occupied_structure,livestock,public_byway,surface_water_supply_area
tracking_#,,,,,,,
400476365,OTHER,FEE,0.0,0.0,0.0,0.0,0.0
400579956,CROP LAND,FEE,1.0,0.0,0.0,0.0,1.0
400584981,NON-CROP LAND,FEE,1.0,0.0,0.0,0.0,0.0
400596597,NON-CROP LAND,OTHER (SPECIFY),0.0,0.0,0.0,0.0,0.0
400597054,NON-CROP LAND,FEE,0.0,0.0,0.0,0.0,0.0


In [16]:
for col in spill_df_incentives.columns:
    print('{} : {}'.format(col,spill_df_incentives[col].unique()))

current_land_use : ['OTHER                              '
 'CROP LAND                          '
 'NON-CROP LAND                      ']
surface_owner : ['FEE' 'OTHER (SPECIFY)' 'FEDERAL' 'STATE' 'TRIBAL']
waters_of_the_state : [0. 1.]
residence_/_occupied_structure : [0. 1.]
livestock : [0. 1.]
public_byway : [0. 1.]
surface_water_supply_area : [0. 1.]


In [17]:
spill_df_incentives['surface_owner'] = spill_df_incentives['surface_owner'].replace('OTHER (SPECIFY)','OTHER')

/Users/alexanderparker/anaconda3/envs/metis/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [18]:
for col in spill_df_incentives.columns:
    print('{} : {}'.format(col,spill_df_incentives[col].unique()))

current_land_use : ['OTHER                              '
 'CROP LAND                          '
 'NON-CROP LAND                      ']
surface_owner : ['FEE' 'OTHER' 'FEDERAL' 'STATE' 'TRIBAL']
waters_of_the_state : [0. 1.]
residence_/_occupied_structure : [0. 1.]
livestock : [0. 1.]
public_byway : [0. 1.]
surface_water_supply_area : [0. 1.]


In [19]:
for col in spill_df_incentives[['waters_of_the_state','residence_/_occupied_structure','livestock',
                                'public_byway','surface_water_supply_area']].columns:
    spill_df_incentives[col] = pd.to_numeric(spill_df_incentives[col],errors='coerce')

/Users/alexanderparker/anaconda3/envs/metis/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [20]:
spill_df_incentives

,current_land_use,surface_owner,waters_of_the_state,residence_/_occupied_structure,livestock,public_byway,surface_water_supply_area
tracking_#,,,,,,,
400476365,OTHER,FEE,0.0,0.0,0.0,0.0,0.0
400579956,CROP LAND,FEE,1.0,0.0,0.0,0.0,1.0
400584981,NON-CROP LAND,FEE,1.0,0.0,0.0,0.0,0.0
400596597,NON-CROP LAND,OTHER,0.0,0.0,0.0,0.0,0.0
400597054,NON-CROP LAND,FEE,0.0,0.0,0.0,0.0,0.0
400597206,NON-CROP LAND,FEDERAL,0.0,0.0,0.0,0.0,0.0
400597258,OTHER,OTHER,0.0,0.0,0.0,0.0,0.0
400597491,NON-CROP LAND,FEDERAL,0.0,0.0,0.0,0.0,0.0
400597595,NON-CROP LAND,FEE,0.0,0.0,0.0,0.0,0.0


## Spill Info

In [21]:
spill_df_info.head()

,date_of_discovery,facility_type,human_error,equipment_failure,historical_unkown,other
tracking_#,,,,,,
400476365,08/30/2013,WELL,False,False,False,False
400579956,03/26/2014,TANK BATTERY,False,False,False,False
400584981,03/15/2013,TANK BATTERY,False,False,False,False
400596597,04/24/2014,WELL,False,False,False,False
400597054,04/21/2014,PIT,False,False,False,False


In [22]:
for col in spill_df_info.columns:
    print('{} : {}'.format(col,spill_df_info[col].unique()))

date_of_discovery : ['08/30/2013' '03/26/2014' '03/15/2013' ... '07/19/2019' '07/18/2019'
 '07/20/2019']
facility_type : ['WELL' 'TANK BATTERY' 'PIT' 'WELL PAD' 'PIPELINE' 'GAS GATHERING SYSTEM'
 'WATER GATHERING SYSTEM/LINE' 'FLOWLINE' 'PARTIALLY-BURIED VESSEL'
 'OTHER' 'CENTRALIZED EP WASTE MGMT FAC' 'PRODUCED WATER TRANSFER SYSTEM'
 'OIL AND GAS LOCATION' 'GAS GATHERING PIPELINE SYSTEM'
 'OFF-LOCATION FLOWLINE' 'GAS COMPRESSOR STATION' 'GAS PROCESSING PLANT']
human_error : [False  True]
equipment_failure : [False  True]
historical_unkown : [False  True]
other : [False  True]


In [23]:
for col in spill_df_info[['human_error','equipment_failure','historical_unkown','other']].columns:
    spill_df_info[col] = spill_df_info[col].astype(int)

/Users/alexanderparker/anaconda3/envs/metis/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [24]:
spill_df_info.head()

,date_of_discovery,facility_type,human_error,equipment_failure,historical_unkown,other
tracking_#,,,,,,
400476365,08/30/2013,WELL,0,0,0,0
400579956,03/26/2014,TANK BATTERY,0,0,0,0
400584981,03/15/2013,TANK BATTERY,0,0,0,0
400596597,04/24/2014,WELL,0,0,0,0
400597054,04/21/2014,PIT,0,0,0,0


In [25]:
spill_df_info.groupby(['facility_type']).date_of_discovery.count()

facility_type
CENTRALIZED EP WASTE MGMT FAC       28
FLOWLINE                           527
GAS COMPRESSOR STATION               7
GAS GATHERING PIPELINE SYSTEM       19
GAS GATHERING SYSTEM                46
GAS PROCESSING PLANT                 4
OFF-LOCATION FLOWLINE                5
OIL AND GAS LOCATION                52
OTHER                              132
PARTIALLY-BURIED VESSEL            100
PIPELINE                           159
PIT                                 69
PRODUCED WATER TRANSFER SYSTEM      20
TANK BATTERY                      1137
WATER GATHERING SYSTEM/LINE        156
WELL                               456
WELL PAD                           432
Name: date_of_discovery, dtype: int64

In [26]:
spill_df_info['date_of_discovery'] = pd.to_datetime(spill_df_info['date_of_discovery'])

/Users/alexanderparker/anaconda3/envs/metis/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [27]:
min_date =  spill_df_info['date_of_discovery'].min()

spill_df_info['days_from_first'] = spill_df_info['date_of_discovery'].apply(lambda x: (x-min_date).days)

/Users/alexanderparker/anaconda3/envs/metis/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [28]:
spill_df_info['month'] = spill_df_info['date_of_discovery'].apply (lambda x: x.month)
spill_df_info['season'] = spill_df_info['month'].replace({1: 'winter',
                                                          2: 'winter',
                                                          3: 'spring',
                                                          4: 'spring',
                                                          5: 'spring',
                                                          6: 'spring',
                                                          7: 'summer',
                                                          8: 'summer',
                                                          9: 'summer',
                                                          10: 'summer',
                                                          11: 'winter',
                                                          12: 'winter'
                                                         })

/Users/alexanderparker/anaconda3/envs/metis/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/alexanderparker/anaconda3/envs/metis/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [29]:
spill_df_info = spill_df_info[['season','days_from_first','facility_type','human_error','equipment_failure',
                              'historical_unkown','other']]

## Combine Features

In [75]:
spill_features = pd.concat([spill_df_info,spill_df_strip_predict,spill_df_incentives],axis=1)
spill_features = spill_features.rename(columns = {'oil_spill_volume' : 'oil_spilled',
                                        'condensate_spill_volume' : 'condensate_spilled',
                                        'flow_back_spill_volume' : 'flowback_spilled',
                                        'produced_water_spill_volume' : 'water_spilled',
                                        'e&p_waste_spill_volume' : 'waste_spilled',
                                         'other_waste': 'other_spilled',
                                        'drilling_fluid_spill_volume' : 'drill_fluid_spilled'})

In [76]:
spill_features.head()

,season,days_from_first,facility_type,human_error,equipment_failure,historical_unkown,other,oil_spilled,condensate_spilled,flowback_spilled,water_spilled,waste_spilled,other_spilled,drill_fluid_spilled,current_land_use,surface_owner,waters_of_the_state,residence_/_occupied_structure,livestock,public_byway,surface_water_supply_area
tracking_#,,,,,,,,,,,,,,,,,,,,,
400476365,summer,1679,WELL,0,0,0,0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,OTHER,FEE,0.0,0.0,0.0,0.0,0.0
400579956,spring,1887,TANK BATTERY,0,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,CROP LAND,FEE,1.0,0.0,0.0,0.0,1.0
400584981,spring,1511,TANK BATTERY,0,0,0,0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,NON-CROP LAND,FEE,1.0,0.0,0.0,0.0,0.0
400596597,spring,1916,WELL,0,0,0,0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,NON-CROP LAND,OTHER,0.0,0.0,0.0,0.0,0.0
400597054,spring,1913,PIT,0,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NON-CROP LAND,FEE,0.0,0.0,0.0,0.0,0.0


In [36]:
spill_target = spill_df['more_than_five_barrels_spilled']
spill_target = spill_target.replace({'Y' : 1,
                                    'N' : 0})

In [79]:
def to_pickle(var,name,dir_name='pickles'):
    with open('{}/{}.pickle'.format(dir_name,name), 'wb') as to_write:
        pickle.dump(var, to_write)
    

In [80]:
X = pd.get_dummies(spill_features)
y = spill_target

to_pickle(X,'X_live')
to_pickle(y,'y_live')